In [1]:
import json
import pickle
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
DATA_DIR = Path("misc_data")
RECIPES_DATA = DATA_DIR / "recipe-ingredients-dataset.csv"

In [ ]:
df = pd.read_csv(RECIPES_DATA)[:10000]

In [4]:
df["tags"] = df["tags"].apply(lambda x: json.loads(x.replace('\'', '"').replace('{', '[').replace('}', ']')))
df["search_terms"] = df["search_terms"].apply(lambda x: json.loads(x.replace('\'', '"').replace('{', '[').replace('}', ']')))
df["ingredients"] = df["ingredients"].apply(lambda x: json.loads(x.replace('\'', '"')))

In [5]:
df["tags_str"] = df["tags"].apply(lambda x: " ".join(x))
df["search_terms_str"] = df["search_terms"].apply(lambda x: " ".join(x))
df["ingredients_str"] = df["ingredients"].apply(lambda x: " ".join(x))

In [6]:
df

,Unnamed: 0,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,cuisine,tags_str,search_terms_str,ingredients_str
0,0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","[water, grits, salt, cheddar cheese, garlic, o...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","[time-to-make, course, main-ingredient, prepar...","[diabetic, low-calorie, vegetarian, low-carb, ...",southern_us,time-to-make course main-ingredient preparatio...,diabetic low-calorie vegetarian low-carb side,water grits salt cheddar cheese garlic olive oil
1,1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","[onion, red bell pepper, garlic cloves, large ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","[60-minutes-or-less, time-to-make, course, mai...","[dinner, shrimp]",cajun_creole,60-minutes-or-less time-to-make course main-in...,dinner shrimp,onion red bell pepper garlic cloves large shri...
2,2,41090,black-and-white bean salad,NaN,"[white beans, canned black beans, tomatoes, on...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,"['In a large bowl, combine beans, tomato, onio...","[15-minutes-or-less, time-to-make, course, mai...","[vegetarian, salad, side, dinner, vegan]",italian,15-minutes-or-less time-to-make course main-in...,vegetarian salad side dinner vegan,white beans canned black beans tomatoes onion ...
3,3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"[zucchini, yellow squash, diced tomatoes, onio...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"[weeknight, time-to-make, course, main-ingredi...","[side, vegetarian, italian]",italian,weeknight time-to-make course main-ingredient ...,side vegetarian italian,zucchini yellow squash diced tomatoes onion ga...
4,4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"[beef stew meat, flour, salt, allspice, cinnam...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","[time-to-make, course, main-ingredient, prepar...",[dinner],russian,time-to-make course main-ingredient preparatio...,dinner,beef stew meat flour salt allspice cinnamon bl...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2495,350629,Belgian White Beer Cookies With Orange Icing,I haven't tried this recipe yet but it sounds ...,"[beer, honey, unsalted butter, powdered sugar,...","["""",""2 (12 ounce) bottles belgian style whi...",1 (133 g),12,['Put the beer and honey to a medium saucepan ...,"[60-minutes-or-less, time-to-make, course, pre...","[cookie, dessert]",cajun_creole,60-minutes-or-less time-to-make course prepara...,cookie dessert,beer honey unsalted butter powdered sugar egg ...
2496,2496,397738,Homestyle Lamb Shanks,"I'm a HUGE Triple D (Diner's, Drive-In's & Div...","[lamb shanks, extra virgin olive oil, beef bro...","[""4 lamb shanks"",""4 -6 tablespoons ...",1 (742 g),4,"['The night before, season the lamb shanks wit...","[course, main-ingredient, main-dish, lamb-shee...","[lamb, dinner]",greek,course main-ingredient main-dish lamb-sheep meat,lamb dinner,lamb shanks extra virgin olive oil beef broth ...
2497,2497,416351,Peppermint Patties,These Peppermint Patties are both Vegan and Gl...,"[coconut oil, agave nectar, peppermint oil, da...","[""1/2 cup coconut oil"",""1/4 cup agave ...",1 (29 g),4,"['In a small bowl, combine coconut oil, agave ...","[15-minutes-or-less, time-to-make, course, pre...","[dessert, gluten-free]",brazilian,15-minutes-or-less time-to-make course prepara...,dessert gluten-free,coconut oil agave nectar peppermint oil dark c...
2498,2498,416432,Veggie Crumble Tacos,I sometimes get a craving for good old tacos b...,"[vegg

In [7]:
df["all_tags"] = df["description"] + " " + df["ingredients_str"] + " " + df["tags_str"] + " " + df["search_terms_str"] + " " + df["cuisine"]

In [8]:
df

,Unnamed: 0,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,cuisine,tags_str,search_terms_str,ingredients_str,all_tags
0,0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","[water, grits, salt, cheddar cheese, garlic, o...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","[time-to-make, course, main-ingredient, prepar...","[diabetic, low-calorie, vegetarian, low-carb, ...",southern_us,time-to-make course main-ingredient preparatio...,diabetic low-calorie vegetarian low-carb side,water grits salt cheddar cheese garlic olive oil,"We love grits, this is another good way to ser..."
1,1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","[onion, red bell pepper, garlic cloves, large ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","[60-minutes-or-less, time-to-make, course, mai...","[dinner, shrimp]",cajun_creole,60-minutes-or-less time-to-make course main-in...,dinner shrimp,onion red bell pepper garlic cloves large shri...,"Simple, easy and very tasty for when you are i..."
2,2,41090,black-and-white bean salad,NaN,"[white beans, canned black beans, tomatoes, on...","[""1 cup canned white beans, rinsed and dra...",1 (807 g),1,"['In a large bowl, combine beans, tomato, onio...","[15-minutes-or-less, time-to-make, course, mai...","[vegetarian, salad, side, dinner, vegan]",italian,15-minutes-or-less time-to-make course main-in...,vegetarian salad side dinner vegan,white beans canned black beans tomatoes onion ...,NaN
3,3,60656,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"[zucchini, yellow squash, diced tomatoes, onio...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"[weeknight, time-to-make, course, main-ingredi...","[side, vegetarian, italian]",italian,weeknight time-to-make course main-ingredient ...,side vegetarian italian,zucchini yellow squash diced tomatoes onion ga...,This is a good recipe for weight watchers. It ...
4,4,232047,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"[beef stew meat, flour, salt, allspice, cinnam...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","[time-to-make, course, main-ingredient, prepar...",[dinner],russian,time-to-make course main-ingredient preparatio...,dinner,beef stew meat flour salt allspice cinnamon bl...,This is a fabulous stew that came from one of ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,2495,350629,Belgian White Beer Cookies With Orange Icing,I haven't tried this recipe yet but it sounds ...,"[beer, honey, unsalted butter, powdered sugar,...","["""",""2 (12 ounce) bottles belgian style whi...",1 (133 g),12,['Put the beer and honey to a medium saucepan ...,"[60-minutes-or-less, time-to-make, course, pre...","[cookie, dessert]",cajun_creole,60-minutes-or-less time-to-make course prepara...,cookie dessert,beer honey unsalted butter powdered sugar egg ...,I haven't tried this recipe yet but it sounds ...
2496,2496,397738,Homestyle Lamb Shanks,"I'm a HUGE Triple D (Diner's, Drive-In's & Div...","[lamb shanks, extra virgin olive oil, beef bro...","[""4 lamb shanks"",""4 -6 tablespoons ...",1 (742 g),4,"['The night before, season the lamb shanks wit...","[course, main-ingredient, main-dish, lamb-shee...","[lamb, dinner]",greek,course main-ingredient main-dish lamb-sheep meat,lamb dinner,lamb shanks extra virgin olive oil beef broth ...,"I'm a HUGE Triple D (Diner's, Drive-In's & Div..."
2497,2497,416351,Peppermint Patties,These Peppermint Patties are both Vegan and Gl...,"[coconut oil, agave nectar, peppermint oil, da...","[""1/2 cup coconut oil"",""1/4 cup agave ...",1 (29 g),4,"['In a small bowl, combine co

In [9]:
df.drop(df[df['all_tags'].isnull()].index, inplace=True)
df = df.reset_index()

In [10]:
tfidf = TfidfVectorizer(max_features=5000, stop_words="english", ngram_range=(1,2))

# Transform the data
vectorized_data = tfidf.fit_transform(df['all_tags'].values)
vectorized_dataframe = pd.DataFrame(vectorized_data.toarray(), index=df['all_tags'].index.tolist())
vectorized_dataframe.shape

(2458, 5000)

In [11]:
svd = TruncatedSVD(n_components=3000)

# Fit transform the data
reduced_data = svd.fit_transform(vectorized_dataframe)
reduced_data

array([[ 3.88163494e-01,  2.13517006e-02,  6.45470285e-02, ...,
         1.18521669e-03, -1.21368456e-03, -1.37390611e-04],
       [ 1.84238431e-01, -1.81238147e-01,  1.34367124e-01, ...,
        -1.32321739e-03,  2.21704644e-03, -5.81655790e-04],
       [ 1.70573827e-01, -1.37694931e-01,  8.12150137e-02, ...,
        -2.40735093e-03,  5.01752585e-04, -3.33115425e-03],
       ...,
       [ 1.03760269e-01, -8.20125770e-02, -1.01450223e-01, ...,
         2.30943473e-04,  7.29821298e-04,  5.75177420e-05],
       [ 4.23151634e-01,  1.20186140e-01,  8.88681486e-02, ...,
        -5.39269050e-03, -1.63203246e-03,  3.21845794e-03],
       [ 1.89285260e-01, -1.08306298e-01,  5.24375419e-02, ...,
        -1.01918310e-03, -1.93409820e-04,  4.48017599e-04]])

In [12]:
def recommendation(prompt, k=5):
    target_vector = tfidf.transform([prompt])
    reduced_target_vector = svd.transform(target_vector)

    similarities = cosine_similarity(reduced_target_vector, reduced_data).flatten()
    topk = np.argpartition(similarities, -k)[-k:]
    
    return [(df.iloc[i]["name"]) for i in topk]

In [13]:

recommendation("a type of sandwich that uses one piece continuous piece of bread that cradles a protein a fiber and a sort of dressing. something I don't have to cook")

['Carrot Ginger Tea Sandwiches',
 'Gourmet Egg Salad Sandwich',
 'Low Fat Peanut Butter Chocolate Crunch Bars',
 'Light and Crisp Veggie Sandwich',
 'The Godfather of Grilled Cheese Sandwiches']

In [14]:
dish_prompts = [
    "I'm looking for a dairy-free breakfast dish with a hint of spice; something like a Mexican-style omelet but without cheese.",
    "Can you suggest a gluten-free Italian lunch option, preferably with a lot of fresh vegetables and no meat?",
    "I need a quick Japanese-inspired snack, but I'm allergic to shellfish—something light and vegetarian would be ideal.",
    "I'm craving a hearty vegan dinner that's Mediterranean-inspired, featuring chickpeas or lentils and lots of fresh herbs.",
    "Could you recommend a kid-friendly dessert that's nut-free but still has a touch of chocolate? Maybe something from French cuisine?",
    "I want a light Thai lunch dish, but it has to be free of soy and seafood, with a focus on fresh and crisp ingredients.",
    "I'm in the mood for a comforting American breakfast but need it to be egg-free—something like pancakes or waffles without eggs would be great.",
    "Give me an Indian-inspired dinner option that's mild in spice and free of dairy. I'd love something with a good mix of veggies.",
    "I need a quick snack that's Mediterranean-style and free of gluten and dairy, ideally something I can eat on the go.",
    "I'm looking for a festive dessert for dinner, inspired by Mexican cuisine but without any cinnamon or nuts."
]

In [15]:
for prompt in dish_prompts:
    print(prompt)
    results = recommendation(prompt)
    for i, dish in enumerate(results, start=1):
        print(f"\t{i}:", dish)
    print()

I'm looking for a dairy-free breakfast dish with a hint of spice; something like a Mexican-style omelet but without cheese.
	1: Taco Bake
	2: The Perfect Non-Dairy, No-Sugar Spiced Chai
	3: Best Tasting Base for Enchilidas. 6 Ingredients!
	4: Gluten- and Dairy-Free Chicken Enchiladas
	5: Quinoa Rice Breakfast (Gluten Free, Dairy Free)

Can you suggest a gluten-free Italian lunch option, preferably with a lot of fresh vegetables and no meat?
	1: Asparagus  Omelette Wraps
	2: Almond Snowballs
	3: Paleo Coconut Bread
	4: Italian Herb Salad Dressing
	5: Fun Mini Muffins -Almond Meal- Flourless

I need a quick Japanese-inspired snack, but I'm allergic to shellfish—something light and vegetarian would be ideal.
	1: Carrot and Sesame Furikake
	2: Miso-Garlic Broiled Eggplant
	3: Garnished Rice With Green Tea (Ocha-Zuke)
	4: Japanese Spring Vegetable Ragout
	5: Light and Crisp Veggie Sandwich

I'm craving a hearty vegan dinner that's Mediterranean-inspired, featuring chickpeas or lentils and l

In [16]:
# save
with open('models/recipe_svd.pkl','wb') as f:
    pickle.dump(svd,f)

# load
with open('models/recipe_svd.pkl', 'rb') as f:
    svd = pickle.load(f)

In [17]:
# save
with open('models/recipe_vectors.pkl','wb') as f:
    pickle.dump(reduced_data,f)

# load
with open('models/recipe_vectors.pkl', 'rb') as f:
    reduced_data = pickle.load(f)

In [18]:
# save
with open('vectorizers/recipe_vectorizer.pkl','wb') as f:
    pickle.dump(tfidf,f)

# load
with open('vectorizers/recipe_vectorizer.pkl', 'rb') as f:
    tfidf = pickle.load(f)

In [19]:
df = df.drop(columns=["index", "id", "Unnamed: 0"])

In [20]:
df.to_csv("data/vectorized_recipes.csv")

In [21]:
df

,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,cuisine,tags_str,search_terms_str,ingredients_str,all_tags
0,Grilled Garlic Cheese Grits,"We love grits, this is another good way to ser...","[water, grits, salt, cheddar cheese, garlic, o...","[""4 cups water"",""1 cup uncooked old f...",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly...","[time-to-make, course, main-ingredient, prepar...","[diabetic, low-calorie, vegetarian, low-carb, ...",southern_us,time-to-make course main-ingredient preparatio...,diabetic low-calorie vegetarian low-carb side,water grits salt cheddar cheese garlic olive oil,"We love grits, this is another good way to ser..."
1,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are i...","[onion, red bell pepper, garlic cloves, large ...","[""1 medium onion, chopped coarse "",""1 m...",1 (366 g),4,"['In a food processor, pulse the onion, red pe...","[60-minutes-or-less, time-to-make, course, mai...","[dinner, shrimp]",cajun_creole,60-minutes-or-less time-to-make course main-in...,dinner shrimp,onion red bell pepper garlic cloves large shri...,"Simple, easy and very tasty for when you are i..."
2,Crock Pot Italian Zucchini,This is a good recipe for weight watchers. It ...,"[zucchini, yellow squash, diced tomatoes, onio...","[""2 zucchini, sliced "",""2 small yel...",1 (244 g),4,['Put all ingredients in the crock pot and coo...,"[weeknight, time-to-make, course, main-ingredi...","[side, vegetarian, italian]",italian,weeknight time-to-make course main-ingredient ...,side vegetarian italian,zucchini yellow squash diced tomatoes onion ga...,This is a good recipe for weight watchers. It ...
3,Beef Stew With Dried Cherries,This is a fabulous stew that came from one of ...,"[beef stew meat, flour, salt, allspice, cinnam...","[""3 lbs beef stew meat"",""3 tablespoons ...",1 (358 g),8,"['Preheat oven to 350°F.', ""Cut beef into 1 in...","[time-to-make, course, main-ingredient, prepar...",[dinner],russian,time-to-make course main-ingredient preparatio...,dinner,beef stew meat flour salt allspice cinnamon bl...,This is a fabulous stew that came from one of ...
4,Hot Sweet Almond Brittle,This is one of our standard holiday gift recip...,"[slivered almonds, cider vinegar, sugar, sugar...","[""12 ounces slivered almonds"",""1/4 cup ...",1 (832 g),1,['Preheat oven to 375°F Place almonds in sing...,"[time-to-make, course, preparation, desserts, ...",[dessert],indian,time-to-make course preparation desserts candy...,dessert,slivered almonds cider vinegar sugar sugar sal...,This is one of our standard holiday gift recip...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2453,Belgian White Beer Cookies With Orange Icing,I haven't tried this recipe yet but it sounds ...,"[beer, honey, unsalted butter, powdered sugar,...","["""",""2 (12 ounce) bottles belgian style whi...",1 (133 g),12,['Put the beer and honey to a medium saucepan ...,"[60-minutes-or-less, time-to-make, course, pre...","[cookie, dessert]",cajun_creole,60-minutes-or-less time-to-make course prepara...,cookie dessert,beer honey unsalted butter powdered sugar egg ...,I haven't tried this recipe yet but it sounds ...
2454,Homestyle Lamb Shanks,"I'm a HUGE Triple D (Diner's, Drive-In's & Div...","[lamb shanks, extra virgin olive oil, beef bro...","[""4 lamb shanks"",""4 -6 tablespoons ...",1 (742 g),4,"['The night before, season the lamb shanks wit...","[course, main-ingredient, main-dish, lamb-shee...","[lamb, dinner]",greek,course main-ingredient main-dish lamb-sheep meat,lamb dinner,lamb shanks extra virgin olive oil beef broth ...,"I'm a HUGE Triple D (Diner's, Drive-In's & Div..."
2455,Peppermint Patties,These Peppermint Patties are both Vegan and Gl...,"[coconut oil, agave nectar, peppermint oil, da...","[""1/2 cup coconut oil"",""1/4 cup agave ...",1 (29 g),4,"['In a small bowl, combine coconut oil, agave ...","[15-minutes-or-less, time-to-make, course, pre...","